# Summary Statistics


In [1]:
import sqlite3

import httpx
import pandas as pd

from litreview import ClinicalTrials

# Make pandas display numbers in non-scientific notation
pd.set_option("display.float_format", lambda x: f"{x:0.1f}")

## Retrieve Integer Fields


In [2]:
API_BASE = "https://clinicaltrials.gov/api/v2"
API_FIELD_VALUES = API_BASE + "/stats/field/values"

response = httpx.get(API_FIELD_VALUES, params={"types": "INTEGER|NUMBER"})
response.raise_for_status()
data = response.json()
integer_fields = [field["piece"] for field in data]
integer_fields

['DispFirstPostYear',
 'DispFirstSubmitYear',
 'EnrollmentCount',
 'EventGroupDeathsNumAffected',
 'EventGroupDeathsNumAtRisk',
 'EventGroupOtherNumAffected',
 'EventGroupOtherNumAtRisk',
 'EventGroupSeriousNumAffected',
 'EventGroupSeriousNumAtRisk',
 'LargeDocSize',
 'LastUpdatePostYear',
 'LastUpdateSubmitYear',
 'NumArmGroupInterventionNames',
 'NumArmGroups',
 'NumAvailIPDs',
 'NumBaselineCategories',
 'NumBaselineClasses',
 'NumBaselineDenoms',
 'NumBaselineGroups',
 'NumBaselineMeasureDenoms',
 'NumBaselineMeasurements',
 'NumBaselineMeasures',
 'NumCentralContacts',
 'NumCollaborators',
 'NumCollaboratorsPlusLead',
 'NumConditionAncestors',
 'NumConditionBrowseBranches',
 'NumConditionBrowseLeafs',
 'NumConditionMeshes',
 'NumConditions',
 'NumDesignWhoMaskeds',
 'NumEventGroups',
 'NumEvents',
 'NumFlowAchievements',
 'NumFlowDropWithdraws',
 'NumFlowGroups',
 'NumFlowMilestones',
 'NumFlowPeriods',
 'NumFlowReasons',
 'NumIPDSharingInfoTypes',
 'NumInterventionAncestors',
 'N

## Load Data from Database


In [3]:
db_connection = sqlite3.connect("../clinical_trials.db")
trials = ClinicalTrials(
    connection=db_connection, schema_directory="../files/schema.json"
)

In [6]:
def get_summary_stats(schema, table="Study"):
    df = pd.DataFrame()
    fields = []

    for key, value in schema.items():
        if isinstance(value, dict):
            df = pd.concat([df, get_summary_stats(value, table=key)], axis=1)
        else:
            if key in integer_fields:
                fields.append(key)

    if fields:
        print(fields)
        query = f"SELECT {', '.join(fields)} FROM {table}"
        current_df = pd.read_sql(query, db_connection).describe()
        df = pd.concat([df, current_df], axis=1)

    return df

In [7]:
df = get_summary_stats(trials._schema)
print(len(df.columns))
df

['EventGroupDeathsNumAffected', 'EventGroupDeathsNumAtRisk', 'EventGroupSeriousNumAffected', 'EventGroupSeriousNumAtRisk', 'EventGroupOtherNumAffected', 'EventGroupOtherNumAtRisk']
['SeriousEventStatsNumAffected', 'SeriousEventStatsNumAtRisk', 'SeriousEventStatsNumEvents']
['OtherEventStatsNumAffected', 'OtherEventStatsNumAtRisk', 'OtherEventStatsNumEvents']
['LargeDocSize']
['SubmissionMCPReleaseN']
['MinimumAge', 'MaximumAge', 'EnrollmentCount', 'TargetDuration', 'NPtrsToThisExpAccNCTId']
16


,EventGroupDeathsNumAffected,EventGroupDeathsNumAtRisk,EventGroupSeriousNumAffected,EventGroupSeriousNumAtRisk,EventGroupOtherNumAffected,EventGroupOtherNumAtRisk,SeriousEventStatsNumAffected,SeriousEventStatsNumAtRisk,SeriousEventStatsNumEvents,OtherEventStatsNumAffected,OtherEventStatsNumAtRisk,OtherEventStatsNumEvents,LargeDocSize,SubmissionMCPReleaseN,EnrollmentCount,NPtrsToThisExpAccNCTId
count,129.0,129.0,285.0,283.0,285.0,283.0,4823.0,4823.0,1007.0,10232.0,10232.0,2942.0,110.0,4.0,982.0,1.0
mean,2.3,47.2,7.6,79.7,24.6,64.0,0.7,188.2,1.4,2.5,41.8,2.1,1234056.9,8.0,2166.2,3.0
std,7.9,73.5,25.1,237.5,47.2,127.2,2.8,421.3,5.1,8.1,57.5,6.0,3236388.4,2.9,48422.6,NaN
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52865.0,5.0,0.0,3.0
25%,0.0,10.0,0.0,10.0,0.0,10.0,0.0,12.0,0.0,0.0,6.0,0.0,252063.5,5.8,30.0,3.0
50%,0.0,21.0,0.0,25.0,6.0,24.0,0.0,60.0,1.0,0.0,20.0,1.0,412523.0,8.0,73.5,3.0
75%,1.0,47.0,3.0,74.0,25.0,73.0,1.0,228.0,1.0,1.0,59.0,2.0,856880.5,10.2,200.0,3.0
max,48.0,373.0,219.0,3278.0,331.0,1682.0,101.0,3278.0,94.0,189.0,522.0,96.0,22463211.0,11.0,1505647.0,3.0
